In [5]:
from glob import glob

for g in glob("./data/*.pdf"):
    print(g)
    

./data/2040_seoul_plan.pdf
./data/OneNYC_2050_Strategic_Plan.pdf


In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


def read_pdf_and_split_text(pdf_path, chunk_size=1000, chunk_overlap=100):
    """
    pdf 파일을 읽어서 텍스트로 변환하고, 텍스트를 잘게 쪼개서 리스트로 반환합니다.

    매개변수:
        pdf_path: str
            텍스트로 변환할 pdf 파일의 경로
        chunk_size: int
            쪼갤 텍스트의 최대 길이
        chunk_overlap: int
            쪼개진 텍스트 조각 간의 겹치는 길이
    """

    print(f"PDF : {pdf_path} ---------------")

    pdf_loader = PyPDFLoader(pdf_path)
    data_from_pdf = pdf_loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    splits = text_splitter.split_documents(data_from_pdf)

    print(f"Number of splits: {len(splits)}\n")

    return splits

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
embedding = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=OPENAI_API_KEY,
)

persist_directory = "../chroma_store"

if os.path.exists(persist_directory):
    print("Loading existing Chroma store")
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding,
    )
else:
    print("Creating new Chroma store")
    vectorstore = None

    for g in glob("./data/*.pdf"):
        chunks = read_pdf_and_split_text(g)

        # 100개씩 나눠서 저장
        for i in range(0, len(chunks), 100):
            if vectorstore is None:
                vectorstore = Chroma.from_documents(
                    documents=chunks[i:i+100],
                    embedding=embedding,
                    persist_directory=persist_directory,
                )
            else:
                vectorstore.add_documents(
                    documents=chunks[i:i+100],
                )



Loading existing Chroma store


In [10]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

chunks = retriever.invoke("서울 온실가스 저감 계획")

for chunk in chunks:
    print(chunk.metadata)
    print(chunk.page_content)

{'page': 45, 'total_pages': 205, 'creationdate': '2024-12-12T18:16:11+09:00', 'producer': 'Hancom PDF 1.3.0.542', 'source': './data/2040_seoul_plan.pdf', 'moddate': '2024-12-12T18:16:11+09:00', 'page_label': '46', 'author': 'SI', 'creator': 'Hwp 2020 11.0.0.5178', 'pdfversion': '1.4'}
기후변화에 대응하기 위한 온실가스 적극 감축 및 녹색 인프라 확충 등을 강조하고 있다.Ÿ2040 서울도시기본계획의 부문별 전략계획과 공간계획에서는 서울의 글로벌 경쟁력 강화, 광역화에 따른 수도권 내 교통, 산업 등 네트워크 연계 강화, 기후위기에 대응하기 위한 지속가능한 서울로의 전환 등 수도권정비기본계획의 주요 내용을 적극적으로 반영하여 수립하였다.
{'author': 'SI', 'producer': 'Hancom PDF 1.3.0.542', 'moddate': '2024-12-12T18:16:11+09:00', 'source': './data/2040_seoul_plan.pdf', 'page_label': '64', 'total_pages': 205, 'pdfversion': '1.4', 'creator': 'Hwp 2020 11.0.0.5178', 'creationdate': '2024-12-12T18:16:11+09:00', 'page': 63}
56제2장 미래상과 목표
6. 미래위기를 준비하는, ‘탄소중립 안전도시 구축’1) 배경전(全) 지구적인 기후변화에 대응하기 위한 대도시 차원의 대응 필요Ÿ서울시 2017년 온실가스 배출량은 46,685천 톤CO2eq로 2005년 배출량에 비해 5.6%(276만 톤CO2eq) 감소하였으며, 서울의 1인당 온실가스 배출량은 4.7CO2eq로 국가의 13.8톤CO2eq에 비해 낮은 수준이다. 또한 2014년 기준

In [11]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)
model.invoke("안녕하세요!")

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 10, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CCRTCzKs83xWFfnFIkTwF3eRrIzEm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ca6ae16e-1d4d-4090-a31f-7ac53f746398-0', usage_metadata={'input_tokens': 10, 'output_tokens': 10, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})